# Max-Voting

### Alistando el modelo


In [1]:
import os
import pandas as pd

In [2]:
#os.chdir(".../Chapter 2")
#os.getcwd()

#### Download the dataset Cryotherapy.csv from the github location and copy the same to your working directory. Let's read the dataset.

In [3]:
cryotherapy_data = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/Cryotherapy.csv")

#### Let's take a glance at the data with the below code:

In [4]:
cryotherapy_data.head(91)

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0
...,...,...,...,...,...,...,...
85,2,34,12.00,3,3,95,0
86,2,20,3.50,6,1,75,1
87,2,35,8.25,8,3,100,0
88,1,24,10.75,10,1,20,1


In [5]:
cryotherapy_data.shape

(90, 7)

### How to do it...

#### We import the required libraries for building the decision tree, support vector machines and logistic regression models. We also import VotingClassifier for max voting

In [6]:
# Import required libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

#### We move onto building our feature set and creating our train & test dataset

In [7]:
# We create train & Test sample from our dataset
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

# create feature & response variables
feature_columns = ['sex', 'age', 'Time', 'Number_of_Warts', 'Type', 'Area']
X = cryotherapy_data[feature_columns]
Y = cryotherapy_data['Result_of_Treatment']

In [8]:
# Create train & test sets
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.20, random_state=1)

### Hard Voting

#### We build our models with decision tree, support vector machines and logistic regression algorithms

In [9]:
# create the sub models
estimators = []

dt_model = DecisionTreeClassifier(random_state=1)
estimators.append(('DecisionTree', dt_model))

svm_model = SVC(random_state=1)
estimators.append(('SupportVector', svm_model))

logit_model = LogisticRegression(random_state=1)
estimators.append(('Logistic Regression', logit_model))

#dt_model.fit(X_train,Y_train)
#svm_model.fit(X_train,Y_train)
#knn_model.fit(X_train,Y_train)

#### We build individual models with each of the classifiers we have chosen

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

for each_estimator in (dt_model, svm_model, logit_model):
    each_estimator.fit(X_train, Y_train)
    Y_pred = each_estimator.predict(X_test)
    print(each_estimator.__class__.__name__, accuracy_score(Y_test, Y_pred))
    print(confusion_matrix(Y_test, Y_pred))

DecisionTreeClassifier 0.8333333333333334
[[8 2]
 [1 7]]
SVC 0.4444444444444444
[[ 0 10]
 [ 0  8]]
LogisticRegression 0.9444444444444444
[[9 1]
 [0 8]]


#### We proceed to ensemble our models and use VotingClassifier to score accuracy

In [11]:
# Using VotingClassifier() to build ensemble model with Hard Voting
ensemble_model = VotingClassifier(estimators=estimators, voting='hard')
ensemble_model.fit(X_train,Y_train)
predicted_labels = ensemble_model.predict(X_test)            
print("Classifier Accuracy using Hard Voting: ", accuracy_score(Y_test, predicted_labels))
print(confusion_matrix(Y_test, predicted_labels))

Classifier Accuracy using Hard Voting:  0.8333333333333334
[[7 3]
 [0 8]]


### Soft Voting

#### The below code creates an ensemble using soft voting:

In [12]:
# create the sub models
estimators = []

dt_model = DecisionTreeClassifier(random_state=1)
estimators.append(('DecisionTree', dt_model))

svm_model = SVC(random_state=1, probability=True)
estimators.append(('SupportVector', svm_model))

logit_model = LogisticRegression(random_state=1)
estimators.append(('Logistic Regression', logit_model))

for each_estimator in (dt_model, svm_model, logit_model):
    each_estimator.fit(X_train, Y_train)
    Y_pred = each_estimator.predict(X_test)
    print(each_estimator.__class__.__name__, accuracy_score(Y_test, Y_pred))
    print(confusion_matrix(Y_test, Y_pred))
    
# Using VotingClassifier() to build ensemble model with Soft Voting
ensemble_model = VotingClassifier(estimators=estimators, voting='soft')
ensemble_model.fit(X_train,Y_train)
predicted_labels = ensemble_model.predict(X_test)
print("Classifier Accuracy using Soft Voting: ", accuracy_score(Y_test, predicted_labels))
print(confusion_matrix(Y_test, predicted_labels))

DecisionTreeClassifier 0.8333333333333334
[[8 2]
 [1 7]]
SVC 0.4444444444444444
[[ 0 10]
 [ 0  8]]
LogisticRegression 0.9444444444444444
[[9 1]
 [0 8]]
Classifier Accuracy using Soft Voting:  0.8888888888888888
[[9 1]
 [1 7]]


# Averaging

#### We download the dataset whitewines.csv from the github location and copy the same to your working directory. Let's read the dataset.

In [13]:
wine_data = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/whitewines.csv")

#### Let's take a glance at the data with the below code

In [14]:
wine_data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.7,0.62,0.24,1.10,0.039,6.0,62.0,0.99340,3.41,0.32,10.400000,5
1,5.7,0.22,0.20,16.00,0.044,41.0,113.0,0.99862,3.22,0.46,8.900000,6
2,5.9,0.19,0.26,7.40,0.034,33.0,123.0,0.99500,3.49,0.42,10.100000,6
3,5.3,0.47,0.10,1.30,0.036,11.0,74.0,0.99082,3.48,0.54,11.200000,4
4,6.4,0.29,0.21,9.65,0.041,36.0,119.0,0.99334,2.99,0.34,10.933333,6


#### We import the required libraries

In [15]:
# Import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

#### We create the response and the feature set

In [16]:
# Create feature and response variable set
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

# create feature & response variables
feature_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',\
                   'chlorides', 'free sulfur dioxide', 'total sulfur dioxide',\
                   'density', 'pH', 'sulphates', 'alcohol']
X = wine_data[feature_columns]
Y = wine_data['quality']

#### We split our data into train & test set

In [17]:
# Create train & test sets
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.30, random_state=1)

#### We build our base regression learners with linear regression, SVR & decision tree

In [18]:
# Build base learners
linreg_model = LinearRegression()
svr_model = SVR()
regressiontree_model = DecisionTreeRegressor()


linreg_model.fit(X_train, Y_train)
svr_model.fit(X_train, Y_train)
regressiontree_model.fit(X_train, Y_train)

DecisionTreeRegressor()

#### Use the base learners to predict on the test data

In [19]:
linreg_predictions = linreg_model.predict(X_test)
svr_predictions = svr_model.predict(X_test)
regtree_predictions = regressiontree_model.predict(X_test)

In [20]:
regtree_predictions

array([5., 5., 6., ..., 6., 6., 6.])

#### We add the predictions and divide by the number of base learners

In [21]:
average_predictions=(linreg_predictions + svr_predictions + regtree_predictions)/3

In [22]:
average_predictions

array([5.24185811, 5.33852934, 5.7580648 , ..., 5.78764674, 6.18555224,
       5.70970165])

In [23]:
Y_test

2414    5
1584    4
3248    6
645     6
3163    5
       ..
1523    7
3634    6
432     5
739     6
2908    5
Name: quality, Length: 1470, dtype: int64

# Weighted Averaging


In [27]:
#os.chdir(".../Chapter 2")
#os.getcwd()

#### We download the Diagnostic Wisconsin Breast Cancer database  wisc_bc_data.csv from the github location and copy the same to your working directory. Let's read the dataset.

In [28]:
cancer_data = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/wisc_bc_data.csv")

#### Let's take a look at the data with the below code

In [29]:
cancer_data.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,0.05955,0.2360,0.6656,1.670,17.43,0.008045,0.011800,0.01683,0.012410,0.01924,0.002248,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,8910251,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,0.06491,0.4505,1.1970,3.430,27.10,0.007470,0.035810,0.03354,0.013650,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,905520,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,0.1967,1.3870,1.342,13.54,0.005158,0.009355,0.01056,0.007483,0.01718,0.002198,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,868871,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,0.06072,0.3384,1.3430,1.851,26.33,0.011270,0.034980,0.02187,0.019650,0.01580,0.003442,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,9012568,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,0.05544,0.1783,0.4125,1.338,17.72,0.005012,0.014850,0.01551,0.009155,0.01647,0.001767,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [30]:
cancer_data.shape

(569, 32)

In [31]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 569 non-null    int64  
 1   diagnosis          569 non-null    object 
 2   radius_mean        569 non-null    float64
 3   texture_mean       569 non-null    float64
 4   perimeter_mean     569 non-null    float64
 5   area_mean          569 non-null    float64
 6   smoothness_mean    569 non-null    float64
 7   compactness_mean   569 non-null    float64
 8   concavity_mean     569 non-null    float64
 9   points_mean        569 non-null    float64
 10  symmetry_mean      569 non-null    float64
 11  dimension_mean     569 non-null    float64
 12  radius_se          569 non-null    float64
 13  texture_se         569 non-null    float64
 14  perimeter_se       569 non-null    float64
 15  area_se            569 non-null    float64
 16  smoothness_se      569 non

In [32]:
cancer_data.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


#### We import the required libraries

In [33]:
# Import required libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#### We create the response and the feature set

In [34]:
# Create feature and response variable set
# We create train & Test sample from our dataset
from sklearn.model_selection import train_test_split

# create feature & response variables
X = cancer_data.iloc[:,2:32]
Y = cancer_data['diagnosis']

In [35]:
# Create train & test sets
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.30, random_state=1)

#### We build our base classifier models

In [36]:
# create the sub models
estimators = []

dt_model = DecisionTreeClassifier()
estimators.append(('DecisionTree', dt_model))

svm_model = SVC(probability=True)
estimators.append(('SupportVector', svm_model))

logit_model = LogisticRegression()
estimators.append(('Logistic Regression', logit_model))

#### We fit our models on the test data

In [37]:
dt_model.fit(X_train, Y_train)
svm_model.fit(X_train, Y_train)
logit_model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [38]:
#### We use the predict_proba() function to predict the class probabilities

In [39]:
dt_predictions = dt_model.predict_proba(X_test)
svm_predictions = svm_model.predict_proba(X_test)
logit_predictions = logit_model.predict_proba(X_test)

#### We assign different weights to each of the models to get our final predictions

In [40]:
weighted_average_predictions=(dt_predictions * 0.3 + svm_predictions * 0.4 + logit_predictions * 0.3)

In [41]:
weighted_average_predictions

array([[2.55984263e-02, 9.74401574e-01],
       [1.01029450e-02, 9.89897055e-01],
       [2.29324794e-01, 7.70675206e-01],
       [5.50796315e-03, 9.94492037e-01],
       [9.86311091e-01, 1.36889088e-02],
       [9.57127878e-01, 4.28721223e-02],
       [9.94240973e-01, 5.75902747e-03],
       [8.07628521e-01, 1.92371479e-01],
       [9.94313015e-01, 5.68698523e-03],
       [9.87834292e-01, 1.21657085e-02],
       [9.90489064e-01, 9.51093602e-03],
       [9.93608864e-01, 6.39113563e-03],
       [9.52034414e-01, 4.79655856e-02],
       [9.90574363e-01, 9.42563678e-03],
       [9.85543551e-01, 1.44564492e-02],
       [2.05154826e-05, 9.99979485e-01],
       [9.88568406e-01, 1.14315941e-02],
       [9.44916746e-01, 5.50832544e-02],
       [4.28022070e-01, 5.71977930e-01],
       [9.95572768e-01, 4.42723227e-03],
       [9.78194290e-01, 2.18057099e-02],
       [9.86512776e-01, 1.34872240e-02],
       [5.63450961e-01, 4.36549039e-01],
       [9.40775484e-01, 5.92245164e-02],
       [9.843358

In [41]:
# Challenge: crea un modelo ponderado con un dataset de tu gusto obteniendo una clasificación binaria, elige 3 modelos de algoritmo diferente de clasificación
